Basado en el artículo de Antal


In [1]:
import networkx as nx
import random

def inicializar_red(num_nodos):
    """
    Crea una red completa (todos los nodos conectados a todos los demás)
    y asigna aleatoriamente relaciones de amistad (+1) o enemistad (-1)
    a cada enlace.

    Args:
        num_nodos (int): El número de nodos en la red.

    Returns:
        nx.Graph: La red inicializada.
    """
    red = nx.Graph()
    red.add_nodes_from(range(num_nodos))

    for i in range(num_nodos):
        for j in range(i + 1, num_nodos):
            # Asigna aleatoriamente +1 (amigo) o -1 (enemigo)
            relacion = random.choice([1, -1])
            red.add_edge(i, j, relacion=relacion)
    return red

def obtener_relacion(red, nodo1, nodo2):
    """
    Obtiene la relación (amistad o enemistad) entre dos nodos.

    Args:
        red (nx.Graph): La red.
        nodo1 (int): El primer nodo.
        nodo2 (int): El segundo nodo.

    Returns:
        int: 1 si son amigos, -1 si son enemigos.
    """
    return red[nodo1][nodo2]['relacion']

def es_triada_balanceada(red, nodo1, nodo2, nodo3):
    """
    Verifica si una tríada (tres nodos conectados entre sí) está balanceada.
    Una tríada está balanceada si el producto de las relaciones entre sus nodos es +1.

    Args:
        red (nx.Graph): La red.
        nodo1 (int): El primer nodo de la tríada.
        nodo2 (int): El segundo nodo de la tríada.
        nodo3 (int): El tercer nodo de la tríada.

    Returns:
        bool: True si la tríada está balanceada, False de lo contrario.
    """
    relacion12 = obtener_relacion(red, nodo1, nodo2)
    relacion13 = obtener_relacion(red, nodo1, nodo3)
    relacion23 = obtener_relacion(red, nodo2, nodo3)

    producto_relaciones = relacion12 * relacion13 * relacion23
    return producto_relaciones == 1

def encontrar_triada_no_balanceada(red):
    """
    Busca una tríada no balanceada en la red.

    Args:
        red (nx.Graph): La red.

    Returns:
        tuple or None: Una tupla de tres nodos que forman una tríada no balanceada,
                       o None si no se encuentra ninguna.
    """
    nodos = list(red.nodes())
    for i in range(len(nodos)):
        for j in range(i + 1, len(nodos)):
            for k in range(j + 1, len(nodos)):
                if red.has_edge(nodos[i], nodos[j]) and \
                   red.has_edge(nodos[i], nodos[k]) and \
                   red.has_edge(nodos[j], nodos[k]):
                    if not es_triada_balanceada(red, nodos[i], nodos[j], nodos[k]):
                        return nodos[i], nodos[j], nodos[k]
    return None

def aplicar_dinamica(red, nodo1, nodo2, nodo3):
    """
    Aplica la dinámica de balance a una tríada no balanceada.
    Cambia la relación entre dos nodos de la tríada para balancearla.
    Se elige aleatoriamente un par de nodos de la tríada y se ajusta su relación.

    Args:
        red (nx.Graph): La red.
        nodo1 (int): El primer nodo de la tríada no balanceada.
        nodo2 (int): El segundo nodo de la tríada no balanceada.
        nodo3 (int): El tercer nodo de la tríada no balanceada.
    """
    pares_nodos = [(nodo1, nodo2), (nodo1, nodo3), (nodo2, nodo3)]
    n1, n2 = random.choice(pares_nodos)

    relacion_actual = obtener_relacion(red, n1, n2)

    # Para balancear la tríada, la nueva relación debe ser el producto de las otras dos
    otro_nodo = list(set([nodo1, nodo2, nodo3]) - set([n1, n2]))[0]
    nueva_relacion = obtener_relacion(red, n1, otro_nodo) * obtener_relacion(red, n2, otro_nodo)

    # Actualiza la relación en la red
    red[n1][n2]['relacion'] = nueva_relacion
    print(f"Se cambió la relación entre {n1} y {n2} a {'amigos' if nueva_relacion == 1 else 'enemigos'}.")

def simular_equilibrio(num_nodos, max_iteraciones=1000):
    """
    Simula la dinámica de balance en la red hasta alcanzar un equilibrio
    o un número máximo de iteraciones.

    Args:
        num_nodos (int): El número de nodos en la red.
        max_iteraciones (int): El número máximo de iteraciones a realizar.

    Returns:
        nx.Graph: La red en estado de equilibrio (o después del máximo de iteraciones).
    """
    red = inicializar_red(num_nodos)
    iteracion = 0
    while iteracion < max_iteraciones:
        triada_no_balanceada = encontrar_triada_no_balanceada(red)
        if triada_no_balanceada:
            print(f"Iteración {iteracion}: Se encontró una tríada no balanceada: {triada_no_balanceada}")
            aplicar_dinamica(red, *triada_no_balanceada)
        else:
            print(f"Se alcanzó el equilibrio en la iteración {iteracion}.")
            break
        iteracion += 1
    else:
        print(f"No se alcanzó el equilibrio después de {max_iteraciones} iteraciones.")
    return red

# Parámetros de la simulación
num_nodos = 6
red_equilibrada = simular_equilibrio(num_nodos)

# Imprimir las relaciones finales en la red equilibrada
print("\nRelaciones finales en la red equilibrada:")
for u, v, data in red_equilibrada.edges(data=True):
    relacion = "amigos" if data['relacion'] == 1 else "enemigos"
    print(f"Nodo {u} y Nodo {v}: {relacion}")

# Verificar si la red final está completamente balanceada
def esta_red_balanceada(red):
    nodos = list(red.nodes())
    for i in range(len(nodos)):
        for j in range(i + 1, len(nodos)):
            for k in range(j + 1, len(nodos)):
                if red.has_edge(nodos[i], nodos[j]) and \
                   red.has_edge(nodos[i], nodos[k]) and \
                   red.has_edge(nodos[j], nodos[k]):
                    if not es_triada_balanceada(red, nodos[i], nodos[j], nodos[k]):
                        return False
    return True

if esta_red_balanceada(red_equilibrada):
    print("\nLa red final está completamente balanceada.")
else:
    print("\nLa red final NO está completamente balanceada (después del máximo de iteraciones).")

Iteración 0: Se encontró una tríada no balanceada: (0, 2, 3)
Se cambió la relación entre 0 y 2 a amigos.
Iteración 1: Se encontró una tríada no balanceada: (0, 1, 2)
Se cambió la relación entre 0 y 2 a enemigos.
Iteración 2: Se encontró una tríada no balanceada: (0, 2, 3)
Se cambió la relación entre 2 y 3 a enemigos.
Iteración 3: Se encontró una tríada no balanceada: (0, 3, 5)
Se cambió la relación entre 3 y 5 a amigos.
Se alcanzó el equilibrio en la iteración 4.

Relaciones finales en la red equilibrada:
Nodo 0 y Nodo 1: amigos
Nodo 0 y Nodo 2: enemigos
Nodo 0 y Nodo 3: amigos
Nodo 0 y Nodo 4: amigos
Nodo 0 y Nodo 5: amigos
Nodo 1 y Nodo 2: enemigos
Nodo 1 y Nodo 3: amigos
Nodo 1 y Nodo 4: amigos
Nodo 1 y Nodo 5: amigos
Nodo 2 y Nodo 3: enemigos
Nodo 2 y Nodo 4: enemigos
Nodo 2 y Nodo 5: enemigos
Nodo 3 y Nodo 4: amigos
Nodo 3 y Nodo 5: amigos
Nodo 4 y Nodo 5: amigos

La red final está completamente balanceada.


Se añade la visualización de la red final para observar las relaciones entre los nodos.

In [3]:
pip install pyvis

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pyvis from https://files.pythonhosted.org/packages/ab/4b/e37e4e5d5ee1179694917b445768bdbfb084f5a59ecd38089d3413d4c70f/pyvis-0.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpickle>=1.4.1 from https://files.pythonhosted.org/packages/86/7c/c06580145924f60342f669f6e71905f838083d00e4b141172a75d22a23fc/jsonpickle-4.0.1-py3-none-any.whl.metadata
Using cached pyvis-0.3.2-py3-none-any.whl (756 kB)
   ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
   -------- ------------------------------- 10.2/46.2 kB ? eta -:--:--
   ----------------- ---------------------- 20.5/46.2 kB 330.3 kB/s eta 0:00:01
   ----------------------------------- ---- 41.0/46.2 kB 326.8 kB/s eta 0:00:01
   ---------------------------------------- 46.2/46.2 kB 327.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:
import networkx as nx
import random
from pyvis.network import Network
import networkx as nx
import random
import json

def inicializar_red(num_nodos):
    """
    Crea una red completa y asigna relaciones aleatorias.
    """
    red = nx.Graph()
    red.add_nodes_from(range(num_nodos))
    for i in range(num_nodos):
        for j in range(i + 1, num_nodos):
            relacion = random.choice([1, -1])
            red.add_edge(i, j, relacion=relacion)
    return red

def obtener_relacion(red, nodo1, nodo2):
    return red[nodo1][nodo2]['relacion']

def es_triada_balanceada(red, nodo1, nodo2, nodo3):
    relacion12 = obtener_relacion(red, nodo1, nodo2)
    relacion13 = obtener_relacion(red, nodo1, nodo3)
    relacion23 = obtener_relacion(red, nodo2, nodo3)
    return relacion12 * relacion13 * relacion23 == 1

def encontrar_triada_no_balanceada(red):
    nodos = list(red.nodes())
    for i in range(len(nodos)):
        for j in range(i + 1, len(nodos)):
            for k in range(j + 1, len(nodos)):
                if red.has_edge(nodos[i], nodos[j]) and \
                   red.has_edge(nodos[i], nodos[k]) and \
                   red.has_edge(nodos[j], nodos[k]):
                    if not es_triada_balanceada(red, nodos[i], nodos[j], nodos[k]):
                        return nodos[i], nodos[j], nodos[k]
    return None

def aplicar_dinamica(red, nodo1, nodo2, nodo3):
    pares_nodos = [(nodo1, nodo2), (nodo1, nodo3), (nodo2, nodo3)]
    n1, n2 = random.choice(pares_nodos)
    otro_nodo = list(set([nodo1, nodo2, nodo3]) - set([n1, n2]))[0]
    nueva_relacion = obtener_relacion(red, n1, otro_nodo) * obtener_relacion(red, n2, otro_nodo)
    red[n1][n2]['relacion'] = nueva_relacion
    print(f"Se cambió la relación entre {n1} y {n2} a {'amigos' if nueva_relacion == 1 else 'enemigos'}.")

def simular_equilibrio(num_nodos, max_iteraciones=1000):
    red = inicializar_red(num_nodos)
    iteracion = 0
    while iteracion < max_iteraciones:
        triada_no_balanceada = encontrar_triada_no_balanceada(red)
        if triada_no_balanceada:
            print(f"Iteración {iteracion}: Tríada no balanceada: {triada_no_balanceada}")
            aplicar_dinamica(red, *triada_no_balanceada)
        else:
            print(f"Equilibrio alcanzado en la iteración {iteracion}.")
            break
        iteracion += 1
    else:
        print(f"No se alcanzó el equilibrio tras {max_iteraciones} iteraciones.")
    return red

def visualizar_red(red, filename="red_social.html"):
    """
    Visualiza la red utilizando pyvis con ajustes de física para mayor estabilidad.

    Args:
        red (nx.Graph): La red a visualizar.
        filename (str): El nombre del archivo HTML para guardar la visualización.
    """
    net = Network(notebook=True, height='100vh', width="100%", select_menu=True, filter_menu=True)

    # Configuración de la física para estabilizar la red
    net.barnes_hut() # Utiliza el algoritmo Barnes-Hut para mejor rendimiento en grafos grandes
    net.show_buttons() # Muestra los botones de física para control interactivo



    for node in red.nodes():
        net.add_node(node, label=str(node))

    for u, v, data in red.edges(data=True):
        color = "blue" if data['relacion'] == 1 else "red"
        weight = 1
        net.add_edge(u, v, color=color, weight=weight)

    net.show(filename, notebook=False)
    print(f"La red se ha visualizado y guardado en '{filename}'")

# Parámetros de la simulación
num_nodos = 50
red_equilibrada = simular_equilibrio(num_nodos)

# Imprimir las relaciones finales
print("\nRelaciones finales:")
for u, v, data in red_equilibrada.edges(data=True):
    relacion = "amigos" if data['relacion'] == 1 else "enemigos"
    print(f"Nodo {u} y Nodo {v}: {relacion}")

# Visualizar la red equilibrada
visualizar_red(red_equilibrada)

Iteración 0: Tríada no balanceada: (0, 1, 2)
Se cambió la relación entre 0 y 2 a amigos.
Iteración 1: Tríada no balanceada: (0, 1, 5)
Se cambió la relación entre 1 y 5 a amigos.
Iteración 2: Tríada no balanceada: (0, 1, 7)
Se cambió la relación entre 0 y 7 a amigos.
Iteración 3: Tríada no balanceada: (0, 1, 13)
Se cambió la relación entre 1 y 13 a amigos.
Iteración 4: Tríada no balanceada: (0, 1, 14)
Se cambió la relación entre 0 y 14 a amigos.
Iteración 5: Tríada no balanceada: (0, 1, 15)
Se cambió la relación entre 0 y 1 a amigos.
Iteración 6: Tríada no balanceada: (0, 1, 2)
Se cambió la relación entre 1 y 2 a amigos.
Iteración 7: Tríada no balanceada: (0, 1, 3)
Se cambió la relación entre 1 y 3 a enemigos.
Iteración 8: Tríada no balanceada: (0, 1, 4)
Se cambió la relación entre 0 y 1 a enemigos.
Iteración 9: Tríada no balanceada: (0, 1, 2)
Se cambió la relación entre 1 y 2 a enemigos.
Iteración 10: Tríada no balanceada: (0, 1, 3)
Se cambió la relación entre 1 y 3 a amigos.
Iteración

Se añade la función para eliminar los enlaces entre nodos enemigos, este recobra la topología vista en las redes de corrupción, donde únicamente están enlazados los nodos amigos.

In [ ]:
import networkx as nx
import random
from pyvis.network import Network
import networkx as nx
import random
import json

def inicializar_red(num_nodos):
    """
    Crea una red completa y asigna relaciones aleatorias.
    """
    red = nx.Graph()
    red.add_nodes_from(range(num_nodos))
    for i in range(num_nodos):
        for j in range(i + 1, num_nodos):
            relacion = random.choice([1, -1])
            red.add_edge(i, j, relacion=relacion)
    return red

def obtener_relacion(red, nodo1, nodo2):
    return red[nodo1][nodo2]['relacion']

def es_triada_balanceada(red, nodo1, nodo2, nodo3):
    relacion12 = obtener_relacion(red, nodo1, nodo2)
    relacion13 = obtener_relacion(red, nodo1, nodo3)
    relacion23 = obtener_relacion(red, nodo2, nodo3)
    return relacion12 * relacion13 * relacion23 == 1

def encontrar_triada_no_balanceada(red):
    nodos = list(red.nodes())
    for i in range(len(nodos)):
        for j in range(i + 1, len(nodos)):
            for k in range(j + 1, len(nodos)):
                if red.has_edge(nodos[i], nodos[j]) and \
                   red.has_edge(nodos[i], nodos[k]) and \
                   red.has_edge(nodos[j], nodos[k]):
                    if not es_triada_balanceada(red, nodos[i], nodos[j], nodos[k]):
                        return nodos[i], nodos[j], nodos[k]
    return None

def aplicar_dinamica(red, nodo1, nodo2, nodo3):
    pares_nodos = [(nodo1, nodo2), (nodo1, nodo3), (nodo2, nodo3)]
    n1, n2 = random.choice(pares_nodos)
    otro_nodo = list(set([nodo1, nodo2, nodo3]) - set([n1, n2]))[0]
    nueva_relacion = obtener_relacion(red, n1, otro_nodo) * obtener_relacion(red, n2, otro_nodo)
    red[n1][n2]['relacion'] = nueva_relacion
    print(f"Se cambió la relación entre {n1} y {n2} a {'amigos' if nueva_relacion == 1 else 'enemigos'}.")

def simular_equilibrio(num_nodos, max_iteraciones=100):
    red = inicializar_red(num_nodos)
    iteracion = 0
    while iteracion < max_iteraciones:
        triada_no_balanceada = encontrar_triada_no_balanceada(red)
        if triada_no_balanceada:
            print(f"Iteración {iteracion}: Tríada no balanceada: {triada_no_balanceada}")
            aplicar_dinamica(red, *triada_no_balanceada)
        else:
            print(f"Equilibrio alcanzado en la iteración {iteracion}.")
            break
        iteracion += 1
    else:
        print(f"No se alcanzó el equilibrio tras {max_iteraciones} iteraciones.")
    return red

def eliminar_enemigos(red):
    """
    Elimina los enlaces entre nodos que son enemigos.

    Args:
        red (nx.Graph): La red.
    """
    edges_to_remove = []
    for u, v, data in red.edges(data=True):
        if data['relacion'] == -1:
            edges_to_remove.append((u, v))
    red.remove_edges_from(edges_to_remove)
    print("Se eliminaron los enlaces entre enemigos.")

def visualizar_red(red, filename="red_social.html"):
    """
    Visualiza la red utilizando pyvis con ajustes de física para mayor estabilidad.

    Args:
        red (nx.Graph): La red a visualizar.
        filename (str): El nombre del archivo HTML para guardar la visualización.
    """
    net = Network(notebook=True, height='100vh', width="100%", select_menu=True, filter_menu=True)

    # Configuración de la física para estabilizar la red
    net.barnes_hut() # Utiliza el algoritmo Barnes-Hut para mejor rendimiento en grafos grandes
    net.show_buttons() # Muestra los botones de física para control interactivo


    for node in red.nodes():
        net.add_node(node, label=str(node))

    for u, v, data in red.edges(data=True):
        color = "blue" if data['relacion'] == 1 else "red"
        weight = 1
        net.add_edge(u, v, color=color, weight=weight)

    net.show(filename, notebook=False)
    print(f"La red se ha visualizado y guardado en '{filename}'")

# Parámetros de la simulación
num_nodos = 20
red_equilibrada = simular_equilibrio(num_nodos)

# Imprimir las relaciones finales
print("\nRelaciones finales:")
for u, v, data in red_equilibrada.edges(data=True):
    relacion = "amigos" if data['relacion'] == 1 else "enemigos"
    print(f"Nodo {u} y Nodo {v}: {relacion}")

# Eliminar los enlaces de enemistad
eliminar_enemigos(red_equilibrada)

# Visualizar la red equilibrada
visualizar_red(red_equilibrada)

Iteración 0: Tríada no balanceada: (0, 1, 2)
Se cambió la relación entre 1 y 2 a enemigos.
Iteración 1: Tríada no balanceada: (0, 1, 4)
Se cambió la relación entre 0 y 1 a amigos.
Iteración 2: Tríada no balanceada: (0, 1, 2)
Se cambió la relación entre 0 y 2 a enemigos.
Iteración 3: Tríada no balanceada: (0, 1, 3)
Se cambió la relación entre 0 y 3 a amigos.
Iteración 4: Tríada no balanceada: (0, 1, 5)
Se cambió la relación entre 1 y 5 a enemigos.
Iteración 5: Tríada no balanceada: (0, 1, 7)
Se cambió la relación entre 1 y 7 a amigos.
Iteración 6: Tríada no balanceada: (0, 1, 8)
Se cambió la relación entre 1 y 8 a enemigos.
Iteración 7: Tríada no balanceada: (0, 1, 14)
Se cambió la relación entre 1 y 14 a enemigos.
Iteración 8: Tríada no balanceada: (0, 1, 15)
Se cambió la relación entre 1 y 15 a amigos.
Iteración 9: Tríada no balanceada: (0, 1, 16)
Se cambió la relación entre 0 y 16 a enemigos.
Iteración 10: Tríada no balanceada: (0, 1, 17)
Se cambió la relación entre 0 y 17 a amigos.


: 